In [1]:
import numpy
import urllib
import scipy.optimize
import random
import math
from math import exp
from math import log

def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

print "Reading data..."
data = list(parseData("file:train.json"))
train_set = data[0:100000]
valid_set = data[100000:200000]
print "done"

Reading data...
done


In [2]:
users = []
items = []
users_items = {}
items_users = {}
datum_index = 0
for datum in train_set:
    u,i = datum['userID'],datum['businessID']
    if not u in users:
        users.append(u)
    if not i in items:
        items.append(i)
    if not users_items.has_key(u):
        users_items[u] = [(i,datum_index)]
    else:
        users_items[u].append((i,datum_index))
    if not items_users.has_key(i):
        items_users[i] = [(u,datum_index)]
    else:
        items_users[i].append((u,datum_index))
    datum_index += 1

In [3]:
sum_rating = 0
count_rating = 0
for u in users_items.keys():
    for i in users_items[u]:
        sum_rating += train_set[i[1]]['rating']
        count_rating += 1
alpha = float(sum_rating)/count_rating

In [4]:
beta_u = []
for u in users_items.keys():
    count_highrate = 0
    count_rate = 0
    for i in users_items[u]:
        count_rate += 1
        if train_set[i[1]]['rating'] > alpha:
            count_highrate += 1
        else:
            count_highrate -= 1
    beta_u.append(float(count_highrate)/count_rate)

In [5]:
beta_i = []
for i in items_users.keys():
    count_highrate = 0
    count_rate = 0
    for u in items_users[i]:
        count_rate += 1
        if train_set[u[1]]['rating'] > alpha:
            count_highrate += 1
        else:
            count_highrate -= 1
    beta_i.append(float(count_highrate)/count_rate)

In [6]:
def getIndex(a,b):
    count = 0
    for ele in b:
        if ele == a:
            return count
        count += 1

def f(Alpha,Betauser,Betaitem,lam):
    f_err = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            f_err += (Alpha+Betauser[user_index]+Betaitem[item_index]-train_set[i[1]]['rating'])**2 + \
                   lam * (Betauser[user_index]**2 + Betaitem[item_index]**2)            
    return f_err
                    
def fprime_user(Alpha,Betauser,Betaitem,lam):
    fprime_err = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            fprime_err += 2 * (Alpha+Betauser[user_index]+Betaitem[item_index]-train_set[i[1]]['rating']) + \
                       2 * lam * Betauser[user_index]
    return fprime_err

def fprime_item(Alpha,Betauser,Betaitem,lam):
    fprime_err = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            fprime_err += 2 * (Alpha+Betauser[user_index]+Betaitem[item_index]-train_set[i[1]]['rating']) + \
                       2 * lam * Betaitem[item_index]
    return fprime_err

def RMSE(Alpha,Betauser,Betaitem):
    square_err = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            square_err += (Alpha+Betauser[user_index]+Betaitem[item_index]-train_set[i[1]]['rating'])**2
    RMSE = math.sqrt(float(square_err)/len(train_set))
    return RMSE

def iterate(Alpha,Betauser,Betaitem,lam):
    # Update alpha
    sum_for_alpha = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            sum_for_alpha += train_set[i[1]]['rating'] - Betauser[user_index] - Betaitem[item_index]
    Alpha = float(sum_for_alpha) / len(train_set)
    # Update beta_user
    for u in users_items.keys():
        sum_for_betauser = 0
        count_item = 0
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            count_item += 1
            sum_for_betauser += train_set[i[1]]['rating'] - Alpha - Betaitem[item_index]
        Betauser[user_index] = float(sum_for_betauser) / (lam + count_item)
    # Update beta_item
    for i in items_users.keys():
        sum_for_betaitem = 0
        count_user = 0
        item_index = getIndex(i,items)
        for u in items_users[i]:
            user_index = getIndex(u[0],users)
            count_user += 1
            sum_for_betaitem += train_set[u[1]]['rating'] - Alpha - Betauser[user_index]
        Betaitem[item_index] = float(sum_for_betaitem) / (lam + count_user)
    return Alpha,Betauser,Betaitem

In [9]:
print 'Initial RMSE = ' + str(RMSE(alpha,beta_u,beta_i))
for lam in [0,0.01,0.1,1.0,10,100]:
    a = alpha
    betau = list(beta_u)
    betai = list(beta_i)
    print 'lam = ' + str(lam)
    print 'Iter\tRMSE'
    for iter_time in range(7):
        count = iter_time + 1
        a,betau,betai = iterate(a,betau,betai,lam)
        with open('lam-'+str(lam)+'-'+str(count)+'-alpha.txt','w+') as f1:
            f1.write(str(a))
        f1.close()
        with open('lam-'+str(lam)+'-'+str(count)+'-betauser.txt','w+') as f2:
            f2.write(str(betau))
        f2.close()
        with open('lam-'+str(lam)+'-'+str(count)+'-betaitem.txt','w+') as f3:
            f3.write(str(betai))
        f3.close()
        print str(count) + '\t' + str(RMSE(a,betau,betai))

Initial RMSE = 1.34444733684
lam = 0
Iter	RMSE
1	0.620649218243
2	0.558688175333
3	0.550453341898
4	0.548432506641
5	0.547754130308
6	0.547471573116
7	0.547331655258
lam = 0.01
Iter	RMSE
1	0.620271883587
2	0.558621741663
3	0.550464637491
4	0.548463896591
5	0.547789778892
6	0.547507141673
7	0.547366234921
lam = 0.1
Iter	RMSE
1	0.617511760222
2	0.558658011458
3	0.551225106824
4	0.549441083786
5	0.548840973057
6	0.548588461953
7	0.548463654349
lam = 1.0
Iter	RMSE
1	0.616840042629
2	0.578743047209
3	0.575713443033
4	0.57509578092
5	0.574873808236
6	0.574764790434
7	0.574701410554
lam = 10
Iter	RMSE
1	0.711842218998
2	0.700449877587
3	0.699230878129
4	0.698947192731
5	0.698863956818
6	0.698834580785
7	0.6988232198
lam = 100
Iter	RMSE
1	0.837685295769
2	0.819693818162
3	0.819512314519
4	0.819506127566
5	0.819505586306
6	0.81950553046
7	0.819505524466


In [ ]:
Initial RMSE = 1.34444733684
lam = 0
Iter	RMSE
1	0.620649218243
2	0.558688175333
3	0.550453341898
4	0.548432506641
5	0.547754130308
6	0.547471573116
7	0.547331655258
lam = 0.01
Iter	RMSE
1	0.620271883587
2	0.558621741663
3	0.550464637491
4	0.548463896591
5	0.547789778892
6	0.547507141673
7	0.547366234921
lam = 0.1
Iter	RMSE
1	0.617511760222
2	0.558658011458
3	0.551225106824
4	0.549441083786
5	0.548840973057
6	0.548588461953
7	0.548463654349
lam = 1.0
Iter	RMSE
1	0.616840042629
2	0.578743047209
3	0.575713443033
4	0.57509578092
5	0.574873808236
6	0.574764790434
7	0.574701410554
lam = 10
Iter	RMSE
1	0.711842218998
2	0.700449877587
3	0.699230878129
4	0.698947192731
5	0.698863956818
6	0.698834580785
7	0.6988232198
lam = 100
Iter	RMSE
1	0.837685295769
2	0.819693818162
3	0.819512314519
4	0.819506127566
5	0.819505586306
6	0.81950553046
7	0.819505524466